In [2]:
import numpy as np
import os
import pickle
from glob import glob
from tqdm import tqdm
import time
from pprint import pprint as pp
from pathlib import Path
import shutil
import random
from matplotlib import pyplot as plt
import json
import xml.etree.ElementTree as ET

# Read GTSDB Json for "pn"

In [3]:
fname = "../data/GTSDB.json"
preds = json.load(open(fname))
preds = preds['output']['frames']
preds[26]['signs'][0].keys()

dict_keys(['coordinates', 'detection_confidence', 'class'])

In [4]:

for frame in preds:
    for det in frame['signs']:
        if det['class']=='pn':
            print(frame['frame_number'],det['coordinates'])

00259.ppm [533, 405, 14, 16]
00259.ppm [485, 358, 15, 28]
00261.ppm [1162, 310, 34, 39]
00274.ppm [378, 361, 22, 30]
00304.ppm [971, 432, 16, 18]
00544.ppm [920, 453, 12, 15]
00545.ppm [363, 439, 21, 22]
00555.ppm [812, 492, 14, 14]
00575.ppm [649, 412, 16, 18]
00673.ppm [311, 428, 22, 24]
00697.ppm [789, 423, 16, 16]
00777.ppm [935, 475, 25, 34]
00813.ppm [1025, 231, 47, 57]
00841.ppm [890, 392, 17, 19]


# Prepare new gt.txt by parsing xml annotations

In [22]:
dataroot = "/home/ubuntu/Shlok/gtsdb/data/FullIJCNN2013"
xmlfiles = glob(dataroot+"/*.xml")
xmlfiles[:3]


['/home/ubuntu/Shlok/gtsdb/data/FullIJCNN2013/00575.xml',
 '/home/ubuntu/Shlok/gtsdb/data/FullIJCNN2013/00673.xml',
 '/home/ubuntu/Shlok/gtsdb/data/FullIJCNN2013/00259.xml']

## Parsing xml to prepare new gt lines

In [39]:
def parseXML(fname):
    tree = ET.parse(fname)
    root = tree.getroot()
    flname = root.find('filename').text
    objects = root.findall('object')
    # print(objects)
    # print(objects.find('bndbox'))
    ret = []
    for obj in objects:
        clname = obj.find('name').text
        bndbox = obj.find('bndbox')
        bbox = [int(bndbox.find('xmin').text),
            int(bndbox.find('ymin').text),
            int(bndbox.find('xmax').text),
            int(bndbox.find('ymax').text)
        ]
        ret.append((flname, clname, bbox))
    
    return ret

annots = []
for file in xmlfiles: annots+=parseXML(file)
newgt_lines = list(map(lambda x: "{};{};{};{};{};{}".format(x[0], x[2][0],  x[2][1],  x[2][2],  x[2][3], x[1]), annots))
newgt_lines[:3]

['00575.ppm;649;413;666;431;43',
 '00673.ppm;311;430;334;453;44',
 '00259.ppm;485;360;502;387;44']

## Saving new gt file

In [41]:
gt_lines = open(os.path.join(dataroot, "gt.txt")).read().split("\n")[:-1]

outfile = os.path.join(dataroot, "gtnew.txt")
with open(outfile, 'w') as fl:
    for line in gt_lines:
        fl.write("{}\n".format(line))
    for line in newgt_lines:
        fl.write("{}\n".format(line))
